# --- Day 17 Pyroclastic Flow ---

https://adventofcode.com/2022/day/17

## Get Input Data

In [1]:
def parse_input(filename):
    """Parse input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    
    """
    with open(f'../inputs/{filename}.txt') as _file:
        jet_pattern = [int(c.replace("<", "-1").replace(">", "1")) for c in _file.read()]

    return jet_pattern

Replace `">"` with `1` and `"<"` with `-1`, as these will represent moves *right* and moves *left*, respectively.

In [2]:
test_jet_pattern = parse_input("test_jet_pattern")
# test_jet_pattern

### Define `rocks` data structure

In [3]:
import numpy as np

In [4]:
rocks = {
    0 : {
        "struct" : np.ones(((1, 4))),
        "height" : 1,
        "width" : 4
    },
    1: {
        "struct" : np.array(((0, 1, 0), (1, 1, 1), (0, 1, 0))),
        "height" : 3,
        "width" : 3
    },
    2: {
        "struct" : np.array(((1, 1, 1), (0, 0, 1), (0, 0, 1))),
        "height" : 3,
        "width" : 3
    },
    3: {
        "struct" : np.ones((4, 1)),
        "height" : 4,
        "width" : 1        
    },
    4: {
        "struct" : np.ones((2, 2)),
        "height" : 2,
        "width" : 2
    },
}

## Part 1
---

In [5]:
def print_chamber(chamber, i_rock):
    """Debugin"""
    print(f"chamber after rock {i_rock + 1} has dropped:")
    print(np.array_str(np.flipud(chamber))
            .replace("1.", "#")
            .replace("0.", ".")
            .replace(" ", "")
            .replace("[[", "[")
            .replace("]]", "]")
            .replace("#######", "-------")
    )

In [6]:
def solve_part1(rocks, jet_pattern, num_rocks=2022):
    floor = np.ones((1, 7))
    space = np.zeros((3, 7))
    chamber = floor

    jet_counter = 0

    # Start droppin' rocks
    for i in range(num_rocks):
        i_rock = i % len(rocks)
        rock = rocks[i_rock]
        rock_space = np.zeros((rock["height"], 7))

        chamber = np.concatenate((chamber, space, rock_space))

        c, r = 2, chamber.shape[0] - rock_space.shape[0]

        falling = True

        # Start applying jets
        while falling:
            i_jet = jet_counter % len(jet_pattern)
            print(r, c, i_jet, i_rock)

            # Apply lateral shift (if any)
            c = max(0, min(7 - rock["width"], c + jet_pattern[i_jet]))
            chamber_slice = chamber[r:r+rock["height"], c:c+rock["width"]]
            if (chamber_slice + rock["struct"]).flatten().sum() > rock["struct"].sum():
                # Bummed into an already fallen rock, so undo previous lateral shift
                # c -= jet_pattern[i_jet]
                c = max(0, min(7 - rock["width"], c - jet_pattern[i_jet]))

            jet_counter += 1

            # Apply vertical drop (until hit floor/other rocks)
            r -= 1
            chamber_slice = chamber[r:r+rock["height"], c:c+rock["width"]]
            if (chamber_slice + rock["struct"]).flatten().sum() > rock["struct"].sum():
                # We've hit the floor/some portion of an already fallen rock
                # So undo the vertical drop... 
                r += 1

                # ...then add the rock to the chamber...
                chamber[r:r+rock["height"], c:c+rock["width"]] += rock["struct"]
                # Debuggin
                print_chamber(chamber, i_rock)
                                
                # ...then truncate the height of the chamber...
                tower_height = chamber.any(axis=1).sum() - 1  # subtract 1 for the floor! 
                chamber = chamber[:tower_height + 1, :]

                # ...then break ot of while loop, which will initiate a new rock drop
                falling = False

    return tower_height

### Run on Test Data

In [8]:
# [solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=r) for r in range(1, 201)]

In [9]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=22)

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

40

In [10]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=1) == 1

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]


True

In [11]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=2) == 4

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]


True

In [12]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=3) == 6

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]


True

In [13]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=4) == 7

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]


True

In [14]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=5) == 9

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]


True

In [15]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=6) == 10

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

True

In [16]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=7) == 13

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

True

In [17]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=8) == 15

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

True

In [18]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=9) == 17

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

True

In [19]:
solve_part1(rocks, parse_input("test_jet_pattern"), num_rocks=10) == 17

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

True

In [20]:
solve_part1(rocks, parse_input("test_jet_pattern")) #== 3068

4 2 0 0
3 3 1 0
2 3 2 0
1 3 3 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[..####.]
[-------]
5 2 4 1
4 1 5 1
3 2 6 1
2 1 7 1
chamber after rock 2 has dropped:
[.......]
[.......]
[.......]
[...#...]
[..###..]
[...#...]
[..####.]
[-------]
8 2 8 2
7 3 9 2
6 2 10 2
5 1 11 2
4 0 12 2
chamber after rock 3 has dropped:
[.......]
[.......]
[.......]
[.......]
[..#....]
[..#....]
[####...]
[..###..]
[...#...]
[..####.]
[-------]
10 2 13 3
9 3 14 3
8 2 15 3
7 3 16 3
6 4 17 3
5 5 18 3
4 4 19 3
chamber after rock 4 has dropped:
[.......]
[.......]
[.......]
[.......]
[.......]
[.......]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
11 2 20 4
10 1 21 4
9 2 22 4
8 3 23 4
chamber after rock 5 has dropped:
[.......]
[.......]
[.......]
[....##.]
[....##.]
[....#..]
[..#.#..]
[..#.#..]
[#####..]
[..###..]
[...#...]
[..####.]
[-------]
13 2 24 0
12 1 25 0
11 0 26 0
10 0 27 0
chamber after rock 1 has dropped:
[.......]
[.......]
[.......]
[.####.

2802

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data